In [1]:
from scipy.spatial.transform import Rotation as R
from scipy.linalg import block_diag
import numpy as np
import quaternion
import math

ModuleNotFoundError: No module named 'quaternion'

In [ ]:
def rotVbyQ(vec, qt):
    """use quaternion to rotate vector"""
    qn = qt.normalized()
    return (qn * np.quaternion(0, *vec)*qn.inverse()).vec

def qnFromVec(v, theta):
    """convert a vec 3d to quaternion"""
    return np.quaternion(0, *v).normalized()*math.sin(theta/2)+quaternion.one*math.cos(theta/2)

def rmatFromQ(q):
    q = q.normalized()
    return block_diag(R.from_quat(q.vec.tolist()+[q.w,]).as_dcm(),[[1,]])
def tmatFromV(x,y,z):
    return [[1,0,0,x],[0,1,0,y],[0,0,1,z],[0,0,0,1]]
def projFrom(D,realX,realY,widthX,heightY):
    return np.dot([[widthX/realX,0,0,widthX/2],
                   [0,heightY/realY,0,heightY/2],
                   [0,0,1,0],
                   [0,0,0,1]],
                  [[1,0,0,0],
                   [0,1,0,0],
                   [0,0,1,-D],
                   [0,0,1/D,0]])
def world2img(rot,trans,proj,vec):
    if rot is None:
        rot = np.eye(4)
    if trans is None:
        trans = np.eye(4)
    if proj is None:
        proj = np.eye(4)
    vec = np.array(vec).reshape(-1)
    if len(vec)<4:
        vec = np.array(vec.tolist()+[1.,])
    v4 = np.linalg.multi_dot([proj,rot,trans,vec])
    return v4[:3]/v4[-1]


In [2]:
rmat = rmatFromQ(qnFromVec((0,0,1),math.pi/4))
print(rmat)
print(np.dot(rmat,np.array([1,0,0,1])))
print(np.dot(rmat,np.array([0,1,0,1])))
print(np.dot(rmat,np.array([0,0,1,1])))

NameError: name 'rmatFromQ' is not defined

In [3]:
proj = projFrom(2.000,2.460,2.122,4000,3000)  # image is (4000 wid, 3000 high)
v4=np.dot(proj,np.array([0,0.5,5,1]))
print(v4)
print(v4/v4[-1])
proj = projFrom(1.000,1.230,1.061,4000,3000)  # image is (4000 wid, 3000 high)
v4=np.dot(proj,np.array([0,0.5,5,1]))
print(v4)
print(v4/v4[-1])


NameError: name 'projFrom' is not defined

In [4]:
rot = rmatFromQ(qnFromVec((0,1,0),math.radians(45)))
print(world2img(None,None, proj, np.array([0,0,1])))
print(world2img(rot,None, proj, np.array([0,0,1])))
print(world2img(None,None, proj, np.array([np.sqrt(2)/2,0,np.sqrt(2)/2])))
print(world2img(None,None, proj, np.array([1,0,1])))

NameError: name 'rmatFromQ' is not defined

In [133]:
import sympy
from sympy.algebras.quaternion import Quaternion
from sympy import symbols, trigsimp, cos, sin, simplify

In [150]:
x,y,z,q0,q1,q2,q3 = sympy.symbols('x y z q0 q1 q2 q3')
D = sympy.symbols('D')
qsym = Quaternion(q0,q1,q2,q3)
print(qsym.to_rotation_matrix())

Matrix([[-2*(q2**2 + q3**2)/(q0**2 + q1**2 + q2**2 + q3**2) + 1, 2*(-q0*q3 + q1*q2)/(q0**2 + q1**2 + q2**2 + q3**2), 2*(q0*q2 + q1*q3)/(q0**2 + q1**2 + q2**2 + q3**2)], [2*(q0*q3 + q1*q2)/(q0**2 + q1**2 + q2**2 + q3**2), -2*(q1**2 + q3**2)/(q0**2 + q1**2 + q2**2 + q3**2) + 1, 2*(-q0*q1 + q2*q3)/(q0**2 + q1**2 + q2**2 + q3**2)], [2*(-q0*q2 + q1*q3)/(q0**2 + q1**2 + q2**2 + q3**2), 2*(q0*q1 + q2*q3)/(q0**2 + q1**2 + q2**2 + q3**2), -2*(q1**2 + q2**2)/(q0**2 + q1**2 + q2**2 + q3**2) + 1]])


In [154]:
rote_X = qsym*Quaternion(0,1,0,0)*qsym.inverse()
rote_Y = qsym*Quaternion(0,0,1,0)*qsym.inverse()
rote_Z = qsym*Quaternion(0,0,0,1)*qsym.inverse()
rotMsym = block_diag(np.array([[rote_X.b,rote_Y.b,rote_Z.b],
          [rote_X.c,rote_Y.c,rote_Z.c],
          [rote_X.d,rote_Y.d,rote_Z.d],]),[[1,]])

In [199]:
projsym = projFrom(D,1.230,1.061,4000,3000)

In [200]:
imgxyzSym = world2img(rotMsym,np.eye(4),projsym,np.array([x,y,z,1]))

In [201]:
qntest = qnFromVec((1,1,1),math.radians(45))
print(sympy.Matrix(imgxyzSym).subs([(x,0),(y,0),(z,1),
                                    (q0,qntest.w),(q1,qntest.x),
                                    (q2,qntest.y),(q3,qntest.z),
                                    (D,1)]))
print(sympy.Matrix(imgxyzSym).subs([(x,0),(y,0),(z,1),
                                    (q0,qntest.w+0.01),(q1,qntest.x),
                                    (q2,qntest.y),(q3,qntest.z),
                                    (D,1)]))

Matrix([[4044.31310483555], [408.617551505243], [-0.242640687119285]])
Matrix([[4015.63858415236], [418.068207352072], [-0.237165653960366]])


In [202]:
delta = 0.00001
numDif_q0 = (sympy.Matrix(imgxyzSym).subs([(x,0),(y,0),(z,1),
                                    (q0,qntest.w+delta),(q1,qntest.x),
                                    (q2,qntest.y),(q3,qntest.z),
                                    (D,1)]) - sympy.Matrix(imgxyzSym).subs([(x,0),(y,0),(z,1),
                                    (q0,qntest.w),(q1,qntest.x),
                                    (q2,qntest.y),(q3,qntest.z),
                                    (D,1)]))/delta
numDif_q1 = (sympy.Matrix(imgxyzSym).subs([(x,0),(y,0),(z,1),
                                    (q0,qntest.w),(q1,qntest.x+delta),
                                    (q2,qntest.y),(q3,qntest.z),
                                    (D,1)]) - sympy.Matrix(imgxyzSym).subs([(x,0),(y,0),(z,1),
                                    (q0,qntest.w),(q1,qntest.x),
                                    (q2,qntest.y),(q3,qntest.z),
                                    (D,1)]))/delta
numDif_q2 = (sympy.Matrix(imgxyzSym).subs([(x,0),(y,0),(z,1),
                                    (q0,qntest.w),(q1,qntest.x),
                                    (q2,qntest.y+delta),(q3,qntest.z),
                                    (D,1)]) - sympy.Matrix(imgxyzSym).subs([(x,0),(y,0),(z,1),
                                    (q0,qntest.w),(q1,qntest.x),
                                    (q2,qntest.y),(q3,qntest.z),
                                    (D,1)]))/delta
numDif_q3 = (sympy.Matrix(imgxyzSym).subs([(x,0),(y,0),(z,1),
                                    (q0,qntest.w),(q1,qntest.x),
                                    (q2,qntest.y),(q3,qntest.z+delta),
                                    (D,1)]) - sympy.Matrix(imgxyzSym).subs([(x,0),(y,0),(z,1),
                                    (q0,qntest.w),(q1,qntest.x),
                                    (q2,qntest.y),(q3,qntest.z),
                                    (D,1)]))/delta
numDif_x = (sympy.Matrix(imgxyzSym).subs([(x,0+delta),(y,0),(z,1),
                                    (q0,qntest.w),(q1,qntest.x),
                                    (q2,qntest.y),(q3,qntest.z),
                                    (D,1)]) - sympy.Matrix(imgxyzSym).subs([(x,0),(y,0),(z,1),
                                    (q0,qntest.w),(q1,qntest.x),
                                    (q2,qntest.y),(q3,qntest.z),
                                    (D,1)]))/delta
numDif_y = (sympy.Matrix(imgxyzSym).subs([(x,0),(y,0+delta),(z,1),
                                    (q0,qntest.w),(q1,qntest.x),
                                    (q2,qntest.y),(q3,qntest.z),
                                    (D,1)]) - sympy.Matrix(imgxyzSym).subs([(x,0),(y,0),(z,1),
                                    (q0,qntest.w),(q1,qntest.x),
                                    (q2,qntest.y),(q3,qntest.z),
                                    (D,1)]))/delta
numDif_z = (sympy.Matrix(imgxyzSym).subs([(x,0),(y,0),(z,1+delta),
                                    (q0,qntest.w),(q1,qntest.x),
                                    (q2,qntest.y),(q3,qntest.z),
                                    (D,1)]) - sympy.Matrix(imgxyzSym).subs([(x,0),(y,0),(z,1),
                                    (q0,qntest.w),(q1,qntest.x),
                                    (q2,qntest.y),(q3,qntest.z),
                                    (D,1)]))/delta
print(numDif_q0.T)
print(numDif_q1.T)
print(numDif_q2.T)
print(numDif_q3.T)
print(numDif_x.T)
print(numDif_y.T)
print(numDif_z.T)


Matrix([[-2908.20313493896, 953.312389322036, 0.557117648955496]])
Matrix([[2908.28588044860, -7091.60808070806, -1.23147757622177]])
Matrix([[8589.60685218335, 953.312389367511, -1.23147757622177]])
Matrix([[663.134503156470, 2151.88737296899, 0.133237252589180]])
Matrix([[4041.12352375705, 1356.20147045188, -0.479643253378792]])
Matrix([[-2540.33067199089, 3513.57080755292, 0.781151681664149]])
Matrix([[0, -2.27373675443232e-8, 1.24262826085020]])


In [203]:
obJacob = sympy.Matrix(imgxyzSym).jacobian([q0,q1,q2,q3,x,y,z]).T
print(obJacob.subs([(x,0),(y,0),(z,1),
                                    (q0,qntest.w),(q1,qntest.x),
                                    (q2,qntest.y),(q3,qntest.z),
                                    (D,1)]))

Matrix([[-2908.24450714611, 953.320716526020, 0.557127425869392], [2908.24450714610, -7091.55557754806, -1.23144294602546], [8589.53428181346, 953.320716526020, -1.23144294602546], [663.163548010500, 2151.88562737781, 0.133234969064807], [4041.10792559117, 1356.19623570745, -0.479641402028536], [-2540.34664127980, 3513.59289477649, 0.781156592193540], [0, 0, 1.24264068711929]])


In [211]:
rx,ry,px,py = sympy.symbols('rx ry px py')
projsym = projFrom(D,rx,ry,px,py)
imgxyzSym = world2img(rotMsym,np.eye(4),projsym,np.array([x,y,z,1]))
obJacob = sympy.Matrix(imgxyzSym).jacobian([q0,q1,q2,q3,x,y,z]).T
print(obJacob.subs(q0**2 + q1**2 + q2**2 + q3**2,1).subs(1.,1))

Matrix([[D*(px*(x*(q0**2 + q1**2 - q2**2 - q3**2) + y*(-2*q0*q3 + 2*q1*q2) + z*(2*q0*q2 + 2*q1*q3))/rx + px*(x*(-2*q0*q2 + 2*q1*q3) + y*(2*q0*q1 + 2*q2*q3) + z*(q0**2 - q1**2 - q2**2 + q3**2))/(2*D))*(-x*(4*q0**2*q2 - 4*q0*q1*q3 - 2*q2) - y*(-4*q0**2*q1 - 4*q0*q2*q3 + 2*q1) - z*(-2*q0**3 + 2*q0*q1**2 + 2*q0*q2**2 - 2*q0*q3**2 + 2*q0))/(x*(-2*q0*q2 + 2*q1*q3) + y*(2*q0*q1 + 2*q2*q3) + z*(q0**2 - q1**2 - q2**2 + q3**2))**2 + D*(px*(x*(-2*q0**3 - 2*q0*q1**2 + 2*q0*q2**2 + 2*q0*q3**2 + 2*q0) + y*(4*q0**2*q3 - 4*q0*q1*q2 - 2*q3) + z*(-4*q0**2*q2 - 4*q0*q1*q3 + 2*q2))/rx + px*(x*(4*q0**2*q2 - 4*q0*q1*q3 - 2*q2) + y*(-4*q0**2*q1 - 4*q0*q2*q3 + 2*q1) + z*(-2*q0**3 + 2*q0*q1**2 + 2*q0*q2**2 - 2*q0*q3**2 + 2*q0))/(2*D))/(x*(-2*q0*q2 + 2*q1*q3) + y*(2*q0*q1 + 2*q2*q3) + z*(q0**2 - q1**2 - q2**2 + q3**2)), D*(py*(x*(2*q0*q3 + 2*q1*q2) + y*(q0**2 - q1**2 + q2**2 - q3**2) + z*(-2*q0*q1 + 2*q2*q3))/ry + py*(x*(-2*q0*q2 + 2*q1*q3) + y*(2*q0*q1 + 2*q2*q3) + z*(q0**2 - q1**2 - q2**2 + q3**2))/(2*D))*(-x